In [1]:
import time
import random, string
from tqdm import tnrange

dataLen = 10000
keyLen = 100
valueLen = 100

Insert test

In [2]:
import pymysql

class DataToMysql:
    def __init__(self, host, user, passwd, db, port):
        try:
            self.conn = pymysql.connect(host=host, user=user, passwd=passwd, db=db,
                                        port=port, charset='utf8')
            self.cursor = self.conn.cursor()
            self.pk = 0
        except pymysql.Error as e:
            raise e
            
    def getConn(self):
        return self.conn
    
    def dropTable(self, table_name):
        self.cursor.execute("DROP TABLE IF EXISTS %s" % table_name)

    def write(self, table_name, info_dict):
        info_dict['ID'] = str(self.pk)
        self.pk += 1
        sql_key = ''
        sql_value = ''
        for key in info_dict.keys():            
            sql_value = (sql_value + '"' + pymysql.escape_string(info_dict[key]) + '"' + ',')
            sql_key = sql_key + ' ' + key + ','

        try:
            self.cursor.execute(
                "INSERT INTO %s (%s) VALUES (%s)" % (table_name, sql_key[:-1], sql_value[:-1]))
            self.conn.commit()
        except pymysql.Error as e:
            if str(e).split(',')[0].split('(')[1] == "1146":
                sql_key_str = ''
                columnStyle = ' text'
                for key in info_dict.keys():
                    if key == 'ID':
                        continue
                    sql_key_str = sql_key_str + ' ' + key + columnStyle + ','
                self.cursor.execute("CREATE TABLE %s (ID int NOT NULL PRIMARY KEY, %s)" % (table_name, sql_key_str[:-1]))
                self.cursor.execute("INSERT INTO %s (%s) VALUES (%s)" %
                                    (table_name, sql_key[:-1], sql_value[:-1]))
                self.conn.commit()
            else:
                raise
                
    def read(self, table_name, where):
        self.cursor.execute("SELECT * FROM %s WHERE %s" % (table_name, where))
        return self.cursor.fetchall()

In [ ]:
col = [''.join(random.choice(string.ascii_letters) for _ in range(10)) for _ in range(keyLen)]
result = []

mysql = DataToMysql(host='140.124.181.78', port=30398, user='wei840222', passwd='j04rmp4jo3', db='database')
mysql.dropTable('test')
for i in tnrange(dataLen):
    data = {}
    for x in range(keyLen): 
        data[col[x]] = ''.join(random.choice(string.ascii_letters) for _ in range(valueLen))
    start = time.time()
    mysql.write('test', data)
    end = time.time()
    result.append(end-start)
mysql.getConn().close()

/home/aclab/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'database.test'")
  result = self._query(query)


output result to csv

In [ ]:
with open('mysql/nfs_10000_insert.csv', 'w') as f:
    f.write('time\n')
    for r in result:
        f.write(str(r)+'\n')

Download test

In [ ]:
result = []

mysql = DataToMysql(host='140.124.181.78', port=30398, user='wei840222', passwd='j04rmp4jo3', db='database')
for i in tnrange(dataLen):
    start = time.time()
    mysql.read('test', 'ID = %d' % i)
    end = time.time()
    result.append(end-start)
mysql.getConn().close()

In [ ]:
with open('mysql/nfs_10000_read.csv', 'w') as f:
    f.write('time\n')
    for r in result:
        f.write(str(r)+'\n') 